# Quantitative Momentum Startegy
Investing in the sotcks that have increase in price most

#   Library import

In [3]:
import numpy as np
import pandas as pd
import requests 
import math
from scipy import stats
import xlsxwriter

# import Our List stocks

In [5]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [6]:
from secrets import IEX_CLOUD_API_TOKEN

# Making our First call

In [12]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data  = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2918284562692,
 'week52high': 184.68,
 'week52low': 121.01,
 'week52highSplitAdjustOnly': 189.34,
 'week52lowSplitAdjustOnly': 119.6,
 'week52change': 0.2733143418993041,
 'sharesOutstanding': 16743325115,
 'float': 0,
 'avg10Volume': 109417301,
 'avg30Volume': 108274499,
 'day200MovingAvg': 161.53,
 'day50MovingAvg': 173.55,
 'employees': 147876,
 'ttmEPS': 6.24,
 'ttmDividendRate': 0.8847151301082837,
 'dividendYield': 0.005136232109742198,
 'nextDividendDate': '',
 'exDividendDate': '2022-02-02',
 'nextEarningsDate': '2022-04-27',
 'peRatio': 29.02540184599062,
 'beta': 1.430645566335162,
 'maxChangePercent': 65.4861245698304,
 'year5ChangePercent': 4.789014172497365,
 'year2ChangePercent': 1.1976328198902415,
 'year1ChangePercent': 0.2699916469051143,
 'ytdChangePercent': -0.0285271625825776,
 'month6ChangePercent': 0.1822679642003273,
 'month3ChangePercent': 0.1480304527304375,
 'month1ChangePercent': -0.013860395307545953,
 'day30ChangeP

# Parising our API Call

In [13]:
data['year1ChangePercent']

0.2699916469051143

# Batch API call

In [33]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
symbol_group = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(0,len(symbol_group)):
    symbol_strings.append(','.join(symbol_group[i]))        

my_columns = ['Ticker','Price','One-Year Price Return','Number of Share to Buy']

In [54]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string},fb&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
#     print(batch_api_url)
    data = requests.get(batch_api_url).json()
    for symbol in (symbol_string.split(',')):
        try:
            final_dataframe = final_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        data[symbol]['price'],
                        data[symbol]['stats']['year1ChangePercent'],
                        'N/A'
                    ],index = my_columns),
                ignore_index=True
            )
        except:
            print("This Symbol having issue while fetching data->"+str(symbol))
final_dataframe

This Symbol having issue while fetching data->WLTW


,Ticker,Price,One-Year Price Return,Number of Share to Buy
0,A,143.36,0.154629,N/A
1,AAL,17.10,-0.044791,N/A
2,AAP,237.81,0.474898,N/A
3,AAPL,176.89,0.277091,N/A
4,ABBV,145.93,0.362559,N/A
...,...,...,...,...
499,YUM,128.19,0.217306,N/A
500,ZBH,125.30,-0.230688,N/A
501,ZBRA,526.00,0.246208,N/A
502,ZION,72.52,0.545853,N/A


# Removing Low Momemtum stock

In [55]:
final_dataframe.sort_values('One-Year Price Return',ascending=False,inplace=True )
final_dataframe=  final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Share to Buy
0,275,LB,80.36,2.375547,N/A
1,148,DVN,54.30,2.118189,N/A
2,317,MRO,22.50,1.78459,N/A
3,109,COP,93.08,1.187643,N/A
4,344,NUE,116.49,1.158484,N/A
5,160,EOG,116.50,1.144546,N/A
6,175,FANG,138.31,1.112231,N/A
7,299,MCHP,73.47,1.073055,N/A
8,195,FTNT,314.52,1.05686,N/A
9,386,PXD,231.73,0.882716,N/A


# Calulating Number of Share to buy

In [67]:
portfoliosize = float(10000000)
postion_size = portfoliosize/len(final_dataframe)
postion_size
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i,'Number of Share to Buy'] = math.floor(postion_size/final_dataframe.loc[i,'Price'])
final_dataframe

C:\Users\aakash\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Share to Buy
0,275,LB,80.36,2.375547,2488
1,148,DVN,54.30,2.118189,3683
2,317,MRO,22.50,1.78459,8888
3,109,COP,93.08,1.187643,2148
4,344,NUE,116.49,1.158484,1716
5,160,EOG,116.50,1.144546,1716
6,175,FANG,138.31,1.112231,1446
7,299,MCHP,73.47,1.073055,2722
8,195,FTNT,314.52,1.05686,635
9,386,PXD,231.73,0.882716,863


# Build a Better Momemtum Startegy

Finding High Momemtum 

In [122]:
hqm_columns =[
    'Ticker',
    'Price',
    'Number Of Share to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',    
    'One-Month Price Return',
    'One-Month Return Percentile'
    
]
hqm_dataframe = pd.DataFrame(columns=hqm_columns)
hqm_dataframe

for symbol_string in symbol_strings:
        batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string},fb&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
        print(batch_api_url)
        data = requests.get(batch_api_url).json()
        for symbol in symbol_string.split(','):
            try:
                hqm_dataframe = hqm_dataframe.append(
                    pd.Series([
                        symbol,
                        float(data[symbol]['price']),
                        'N/A',
                        data[symbol]['stats']['year1ChangePercent'],
                        'N/A',
                        data[symbol]['stats']['month6ChangePercent'],
                        'N/A',
                        data[symbol]['stats']['month3ChangePercent'],
                        'N/A',    
                        data[symbol]['stats']['month1ChangePercent'],
                        'N/A'                    
                    ],index=hqm_columns                   
                    ),ignore_index=True
                )
            except:
                print("This Symbol having issue while fetching data->"+str(symbol))
hqm_dataframe

https://sandbox.iexapis.com/stable/stock/market/batch?symbols=A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,fb&types=price,stats&token=Tpk_059b97af715d417d9f49f50b51b1c448
https://sandbox.iexapis.com/stable/stock/market/batch?symbols=CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT

,Ticker,Price,Number Of Share to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,148.17,N/A,0.158133,N/A,-0.089837,N/A,-0.099378,N/A,-0.052355,N/A
1,AAL,16.93,N/A,-0.043261,N/A,-0.218367,N/A,-0.253769,N/A,-0.124821,N/A
2,AAP,231.04,N/A,0.468565,N/A,0.115142,N/A,-0.007977,N/A,-0.041436,N/A
3,AAPL,177.14,N/A,0.272437,N/A,0.178217,N/A,0.142512,N/A,-0.013423,N/A
4,ABBV,147.43,N/A,0.358616,N/A,0.255063,N/A,0.215544,N/A,0.047609,N/A
...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,131.34,N/A,0.219197,N/A,-0.057657,N/A,-0.007504,N/A,-0.078119,N/A
500,ZBH,127.65,N/A,-0.238942,N/A,-0.186713,N/A,-0.133501,N/A,-0.057442,N/A
501,ZBRA,526.70,N/A,0.245202,N/A,-0.12097,N/A,-0.17899,N/A,-0.099679,N/A
502,ZION,73.76,N/A,0.555085,N/A,0.35901,N/A,0.105927,N/A,0.077268,N/A


# Calculating Momemtum Percentile

In [125]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]
for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0
hqm_dataframe
for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row,f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'],hqm_dataframe.loc[row,f'{time_period} Price Return'])

hqm_dataframe

,Ticker,Price,Number Of Share to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,148.17,N/A,0.158133,48.412698,-0.089837,23.412698,-0.099378,25.0,-0.052355,43.849206
1,AAL,16.93,N/A,-0.043261,15.674603,-0.218367,5.15873,-0.253769,3.174603,-0.124821,11.706349
2,AAP,231.04,N/A,0.468565,86.507937,0.115142,74.801587,-0.007977,55.15873,-0.041436,48.015873
3,AAPL,177.14,N/A,0.272437,66.269841,0.178217,82.936508,0.142512,91.865079,-0.013423,62.103175
4,ABBV,147.43,N/A,0.358616,78.373016,0.255063,90.873016,0.215544,97.02381,0.047609,84.920635
...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,131.34,N/A,0.219197,56.944444,-0.057657,29.761905,-0.007504,55.357143,-0.078119,30.753968
500,ZBH,127.65,N/A,-0.238942,3.174603,-0.186713,8.730159,-0.133501,15.873016,-0.057442,41.468254
501,ZBRA,526.70,N/A,0.245202,61.111111,-0.12097,17.65873,-0.17899,9.325397,-0.099679,19.246032
502,ZION,73.76,N/A,0.555085,92.65873,0.35901,95.436508,0.105927,85.714286,0.077268,91.269841
